In [ ]:
import os
import platform
import warnings
import itertools
warnings.filterwarnings("ignore", category=DeprecationWarning) 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['OMP_NUM_THREADS'] = '1'

from functools import lru_cache

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import keras.backend as K
import tensorflow_addons as tfa
from scipy import signal
from tensorflow.keras import layers
from tensorflow.keras.initializers import Constant, GlorotUniform

import numpy as np

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

tf.random.set_seed(9)

X_TRAIN = np.load("direction_x_train.npy", mmap_mode="r")
X_TRAIN_PHI = np.load("direction_x_train_phi.npy", mmap_mode="r")
X_LEVEL_TRAIN = np.load("direction_x_level_train.npy", mmap_mode="r")
Y_TRAIN = np.load("direction_y_train.npy", mmap_mode="r")
ROOMS_TRAIN = np.load("direction_room_train.npy", mmap_mode="r")
whistle_length_train = np.load("whistle_length_train.npy", mmap_mode="r")
whistle_gap_length_train = np.load("whistle_gap_length_train.npy", mmap_mode="r")

X_TEST = np.load("direction_x_test.npy", mmap_mode="r")
X_TEST_PHI = np.load("direction_x_test_phi.npy", mmap_mode="r")
X_LEVEL_TEST = np.load("direction_x_level_test.npy", mmap_mode="r")
Y_TEST = np.load("direction_y_test.npy", mmap_mode="r")
ROOMS_TEST = np.load("direction_room_test.npy", mmap_mode="r")
whistle_length_test = np.load("whistle_length_test.npy", mmap_mode="r")
whistle_gap_length_test = np.load("whistle_gap_length_test.npy", mmap_mode="r")

In [7]:
BINS = np.array([2.0, 4.5, 6.75, 9, np.inf])
print(BINS)

[2.   4.5  6.75 9.    inf]


In [8]:
BATCH_SIZE = 32
MIN_WHISTLE_SIZE = 18
GAUSS = signal.gaussian((2*len(BINS))+1, std=0.56)
GAUSS = (GAUSS - np.min(GAUSS))/(np.max(GAUSS) - np.min(GAUSS))
GAUSS = GAUSS.astype(np.float32)

In [155]:
@lru_cache(maxsize=None)
def get_nearest_bin(data, bins):
    tmp_bins = np.copy(bins)
    tmp_bins = np.hstack([0.0, tmp_bins])
    max_error = np.diff(tmp_bins)/2
    bin_centers = tmp_bins[:-1] + max_error
    max_error[-1] = bin_centers[-1] = np.max(Y_TRAIN[:,1])
    error = np.abs(tmp_bins - data)
    scale_error = np.abs(bin_centers - data)
    scale = scale_error[np.digitize(data, bins, right=True)] / max_error[np.digitize(data, bins, right=True)]
    if np.min(scale_error) == 0:
        return None, scale
    else:
        return np.argmin(error) - 1, scale


@lru_cache(maxsize=None)
def get_dampening_mask(value, bins):
    i = np.digitize(value, bins, right=True)
    j, scale = get_nearest_bin(value, bins)
    idxs = np.arange(0, len(bins), 1)
    if j is None:
        return idxs != i, scale
    elif i <= j:
        return idxs < i, scale
    else:
        return idxs > i, scale


@lru_cache(maxsize=None)
def discretization(data, bins, tol=0.01):
    idx = np.digitize(data, BINS, right=True)

    encoded_data = np.zeros(shape=BINS.shape, dtype=np.float32)
    encoded_data = np.copy(GAUSS[len(BINS)-idx:(len(BINS)-idx)+len(BINS)])
    mask, scale = get_dampening_mask(data, bins)
    encoded_data[mask] *= (1 + (tol*scale)) - scale
    mask[idx] = True
    mask = np.invert(mask)
    encoded_data[mask] *= (1 - (tol*scale)) + scale
    return encoded_data


def data_generator(whistle_length, x, y, r, window_size_direct=3, use_dropout=False, use_noise=False, use_scale=False, batch_size=BATCH_SIZE, noise=0.001, scale=0.001, broken_mic_probability=20):
    shape = x[0].shape
    whistle_length = itertools.cycle(whistle_length)
    x = itertools.cycle(x)
    y = itertools.cycle(y)
    r = itertools.cycle(r)

    while True:
        X = []
        DRR = []
        Y_dist = []
        ROOMS = []

        for _ in range(batch_size):

            # skip too short whistles
            length = next(whistle_length)
            while length < MIN_WHISTLE_SIZE:
                for _ in range(length):
                    next(x)
                    next(y)
                    next(r)
                length = next(whistle_length)
            
            # build spectrogram and drr
            spectrogram = []
            dist = None
            direct = np.zeros(shape=shape)
            reverberation = np.zeros(shape=shape)
            mic_broken_bool = np.random.random() < (broken_mic_probability/100)
            broken_mic = np.random.randint(0, 4)
            if use_scale:
                scale_mask = np.ones(shape[1])
                scale_mask = scale_mask * np.random.normal(loc=0.0, scale=scale)
            for i in range(MIN_WHISTLE_SIZE):
                amp_slice = np.abs(next(x))
                if use_scale:
                    amp_slice = amp_slice + scale_mask
                    amp_slice = np.clip(amp_slice, K.epsilon(), np.inf)
                if use_noise:
                    noise_mask = np.ones(shape)
                    noise_mask = noise_mask * np.random.normal(loc=0.0, scale=noise, size=shape)
                    amp_slice = amp_slice + noise_mask
                    amp_slice = np.clip(amp_slice, K.epsilon(), np.inf)
                if use_dropout and mic_broken_bool:
                    amp_slice[:,broken_mic] = np.ones_like(amp_slice[:,broken_mic]) * K.epsilon()
                spectrogram_slice = 20 * np.log10(amp_slice)
                
                spectrogram.append(spectrogram_slice)

                if i < window_size_direct:
                    direct += amp_slice
                else:
                    reverberation += amp_slice

                dist = next(y)[1]
                room = next(r)
                
            # skip rest of long whistles
            for _ in range(np.abs(length - MIN_WHISTLE_SIZE)):
                next(x)
                next(y)
                next(r)

            X.append(spectrogram)
            DRR.append(20 * np.log10(direct/reverberation))
            Y_dist.append(discretization(dist, tuple(BINS)))
            ROOMS.append(room)
        yield np.asarray(X),  np.asarray(DRR), np.asarray(Y_dist), np.asarray(ROOMS)

GENERATOR = data_generator(whistle_length_train, X_TRAIN, Y_TRAIN, ROOMS_TRAIN, use_dropout=False, use_noise=False, use_scale=False)

In [157]:
idxs, counts = np.unique(np.digitize(Y_TRAIN[:,1], BINS, right=True), return_counts=True)
sum_of_counts = np.sum(counts)
initial_bias_distance = np.zeros(BINS.shape, dtype=np.float32)
for c_idx, idx in enumerate(idxs):
    initial_bias_distance[idx] = counts[c_idx]/sum_of_counts

str_init_bias_distance = np.array2string(initial_bias_distance, precision=4, separator=",", max_line_width=np.inf, floatmode="fixed")
print(f"initial bias distance: \t{str_init_bias_distance}")

initial bias distance: 	[0.3869,0.2901,0.1095,0.1623,0.0512]


In [183]:
def create_backbone2D(shape):
    inputs = layers.Input(shape=shape)
    augmented = layers.GaussianNoise(stddev=5)(inputs)
    dropout = layers.Dropout(rate=0.25, noise_shape=(BATCH_SIZE, 1, 1, 4))(augmented)

    conv = layers.Conv2D(filters=64, kernel_size=(3,1), strides=1, data_format='channels_last', padding="same", use_bias=False, kernel_initializer=GlorotUniform)(dropout)
    leakyReLU = layers.LeakyReLU(alpha=0.3)(conv)
    maxpool = layers.MaxPool2D(pool_size=(1,2), strides=(1,2), data_format='channels_first', padding="same")(leakyReLU)
    maxpool = layers.MaxPool2D(pool_size=(2,1), strides=(2,1), data_format='channels_first', padding="valid")(maxpool)
    maxpool = layers.MaxPool2D(pool_size=(2,1), strides=(2,1), data_format='channels_last', padding="valid")(maxpool)

    conv = layers.Conv2D(filters=32, kernel_size=(3,1), strides=1, data_format='channels_last', padding="same", use_bias=False, kernel_initializer=GlorotUniform)(maxpool)
    leakyReLU = layers.LeakyReLU(alpha=0.3)(conv)
    maxpool = layers.MaxPool2D(pool_size=(1,2), strides=(1,2), data_format='channels_first', padding="same")(leakyReLU)
    maxpool = layers.MaxPool2D(pool_size=(2,1), strides=(2,1), data_format='channels_first', padding="valid")(maxpool)
    maxpool = layers.MaxPool2D(pool_size=(2,1), strides=(2,1), data_format='channels_last', padding="valid")(maxpool)

    conv = layers.Conv2D(filters=16, kernel_size=(3,1), strides=1, data_format='channels_last', padding="same", use_bias=False, kernel_initializer=GlorotUniform)(maxpool)
    leakyReLU = layers.LeakyReLU(alpha=0.3)(conv)
    maxpool = layers.MaxPool2D(pool_size=(1,2), strides=(1,2), data_format='channels_first', padding="same")(leakyReLU)
    maxpool = layers.MaxPool2D(pool_size=(2,1), strides=(2,1), data_format='channels_first', padding="valid")(maxpool)
    maxpool = layers.MaxPool2D(pool_size=(2,1), strides=(2,1), data_format='channels_last', padding="valid")(maxpool)

    conv = layers.Conv2D(filters=8, kernel_size=(3,1), strides=1, data_format='channels_last', padding="same", use_bias=False, kernel_initializer=GlorotUniform)(maxpool)
    leakyReLU = layers.LeakyReLU(alpha=0.3)(conv)
    maxpool = layers.MaxPool2D(pool_size=(1,2), strides=(1,2), data_format='channels_first', padding="same")(leakyReLU)
    maxpool = layers.MaxPool2D(pool_size=(2,1), strides=(2,1), data_format='channels_first', padding="valid")(maxpool)
    maxpool = layers.MaxPool2D(pool_size=(2,1), strides=(2,1), data_format='channels_last', padding="valid")(maxpool)

    flatten = layers.Flatten()(maxpool)

    dense = layers.Dense(64, activation="relu", kernel_regularizer=tf.keras.regularizers.L1(0.001), use_bias=False, kernel_initializer=GlorotUniform)(flatten)
    dropout = layers.Dropout(rate=0.2)(dense)
    dense = layers.Dense(32, activation="relu", kernel_regularizer=tf.keras.regularizers.L1(0.001), use_bias=False, kernel_initializer=GlorotUniform)(dropout)
    dropout = layers.Dropout(rate=0.2)(dense)
    dense = layers.Dense(16, activation="relu", kernel_regularizer=tf.keras.regularizers.L1(0.001), use_bias=False, kernel_initializer=GlorotUniform)(dropout)
    features = layers.Dropout(rate=0.2)(dense)

    return tf.keras.Model(inputs=inputs, outputs=features)

def create_backbone1D(shape):
    inputs = layers.Input(shape=shape)
    augmented = layers.GaussianNoise(stddev=0.5)(inputs)
    dropout = layers.Dropout(rate=0.25, noise_shape=(BATCH_SIZE, 1, 4))(augmented)

    conv = layers.Conv1D(filters=64, kernel_size=3, strides=1, padding="same", use_bias=False, kernel_initializer=GlorotUniform)(dropout)
    leakyReLU = layers.LeakyReLU(alpha=0.3)(conv)
    maxpool = layers.MaxPool1D(pool_size=2, strides=2, data_format='channels_first', padding="same")(leakyReLU)
    maxpool = layers.MaxPool1D(pool_size=2, strides=2, data_format='channels_last', padding="valid")(maxpool)

    conv = layers.Conv1D(filters=32, kernel_size=3, strides=1, padding="same", use_bias=False, kernel_initializer=GlorotUniform)(maxpool)
    leakyReLU = layers.LeakyReLU(alpha=0.3)(conv)
    maxpool = layers.MaxPool1D(pool_size=2, strides=2, data_format='channels_first', padding="same")(leakyReLU)
    maxpool = layers.MaxPool1D(pool_size=2, strides=2, data_format='channels_last', padding="valid")(maxpool)

    conv = layers.Conv1D(filters=16, kernel_size=3, strides=1, padding="same", use_bias=False, kernel_initializer=GlorotUniform)(maxpool)
    leakyReLU = layers.LeakyReLU(alpha=0.3)(conv)
    maxpool = layers.MaxPool1D(pool_size=2, strides=2, data_format='channels_first', padding="same")(leakyReLU)
    maxpool = layers.MaxPool1D(pool_size=2, strides=2, data_format='channels_last', padding="valid")(maxpool)

    conv = layers.Conv1D(filters=8, kernel_size=3, strides=1, padding="same", use_bias=False, kernel_initializer=GlorotUniform)(maxpool)
    leakyReLU = layers.LeakyReLU(alpha=0.3)(conv)
    maxpool = layers.MaxPool1D(pool_size=2, strides=2, data_format='channels_first', padding="same")(leakyReLU)
    maxpool = layers.MaxPool1D(pool_size=2, strides=2, data_format='channels_last', padding="valid")(maxpool)

    flatten = layers.Flatten()(maxpool)

    dense = layers.Dense(64, activation="relu", kernel_regularizer=tf.keras.regularizers.L1(0.001), use_bias=False, kernel_initializer=GlorotUniform)(flatten)
    dropout = layers.Dropout(rate=0.2)(dense)
    dense = layers.Dense(32, activation="relu", kernel_regularizer=tf.keras.regularizers.L1(0.001), use_bias=False, kernel_initializer=GlorotUniform)(dropout)
    dropout = layers.Dropout(rate=0.2)(dense)
    dense = layers.Dense(16, activation="relu", kernel_regularizer=tf.keras.regularizers.L1(0.001), use_bias=False, kernel_initializer=GlorotUniform)(dropout)
    features = layers.Dropout(rate=0.2)(dense)

    return tf.keras.Model(inputs=inputs, outputs=features)

def create_model():
    spectrogram = layers.Input(shape=(18, 513, 4))
    drr = layers.Input(shape=(513, 4))

    spectrogram_backbone = create_backbone2D((18, 513, 4))
    spectrogram_features = spectrogram_backbone(spectrogram)

    drr_backbone = create_backbone1D((513, 4))
    drr_features = drr_backbone(drr)

    features = layers.Concatenate()([spectrogram_features, drr_features])

    # Feature refinement
    dense = layers.Dense(32, activation="relu", kernel_regularizer=tf.keras.regularizers.L1(0.001), use_bias=False, kernel_initializer=GlorotUniform)(features)
    dropout = layers.Dropout(rate=0.2)(dense)
    dense = layers.Dense(32, activation="relu", kernel_regularizer=tf.keras.regularizers.L1(0.001), use_bias=False, kernel_initializer=GlorotUniform)(dropout)
    dropout = layers.Dropout(rate=0.2)(dense)
    dense = layers.Dense(32, activation="relu", kernel_regularizer=tf.keras.regularizers.L1(0.001), use_bias=False, kernel_initializer=GlorotUniform)(dropout)
    dropout = layers.Dropout(rate=0.2)(dense)

    # Conclusion
    distance = layers.Dense(5, activation="sigmoid", use_bias=True, bias_initializer=Constant([0.3869,0.2901,0.1095,0.1623,0.0512]), kernel_initializer=GlorotUniform)(dropout)

    return tf.keras.Model(inputs=[spectrogram, drr], outputs=distance)

tf.random.set_seed(9)
GENERATOR = data_generator(whistle_length_train, X_TRAIN, Y_TRAIN, ROOMS_TRAIN)
TEST_GENERATOR = data_generator(whistle_length_test, X_TEST, Y_TEST, ROOMS_TEST)

In [184]:
model = create_model()

In [186]:
SCALE = 1
GAMMA = 2
ALPHA = 0.75

@tf.function
def dist_loss(y_true, y_pred, scale = SCALE):
    # Adapted Focal Loss
    p_t = 1 - (tf.abs(y_true - y_pred) / tf.math.maximum(tf.abs(1 - y_true), tf.abs(0 - y_true)))
    alpha_t = tf.where(tf.equal(y_true, 1.0), ALPHA, 1 - ALPHA)
    error = -alpha_t * tf.math.pow((1 - p_t), GAMMA) * tf.math.log(p_t + K.epsilon())
    error = tf.reduce_mean(tf.reduce_sum(error, axis=-1))

    return scale * error

In [187]:
nadam = tf.optimizers.Nadam(learning_rate=3e-4)
optimizer = nadam

In [188]:
@tf.function
def train_step(spectrogram, drr, distances):
    with tf.GradientTape() as tape:
        pred_distances = model([spectrogram, drr], training=True)

        model_loss = dist_loss(distances, pred_distances)

        grads = tape.gradient(model_loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))        

    return model_loss

In [ ]:
EPOCHS = 2500
PATIENCE = 100
MIN_DELTA = 0.001
TRAIN_STEPS = len(whistle_length_train[whistle_length_train>=MIN_WHISTLE_SIZE])/BATCH_SIZE
TEST_STEPS = len(whistle_length_test[whistle_length_test>=MIN_WHISTLE_SIZE])/BATCH_SIZE
CRITERIA = "mean"

mean_epoch_loss = np.inf
mean_train_epoch_losses = []
mean_validate_epoch_losses = []

median_epoch_loss = np.inf
median_train_epoch_losses = []
median_validate_epoch_losses = []

epoch_losses = []
min_validate_loss = np.inf
used_epochs = EPOCHS
no_improvements = 0
current_validation_loss = np.inf

for epoch in range(EPOCHS):
    if len(epoch_losses) > 0:
        mean_epoch_loss = np.mean(epoch_losses)
        mean_train_epoch_losses.append(mean_epoch_loss)

        median_epoch_loss = np.median(epoch_losses)
        median_train_epoch_losses.append(median_epoch_loss)

        validate_losses = []
        validate_ranges = []
        for step, batch in enumerate(TEST_GENERATOR):
            if step >= TEST_STEPS:
                    break
            spectrogram, drr, distances, rooms = batch
            pred_distances = model.predict([spectrogram, drr])
            distance_loss = dist_loss(distances, pred_distances)
            validate_losses.append(distance_loss)
        mean_validate_epoch_losses.append(np.mean(validate_losses))
        median_validate_epoch_losses.append(np.median(validate_losses))

        if CRITERIA == "median":
            current_validation_loss = np.median(validate_losses)
        else:
            current_validation_loss = np.mean(validate_losses)
        
        if min_validate_loss - current_validation_loss > MIN_DELTA:
            no_improvements = 0
        else:
            no_improvements += 1

        if current_validation_loss < min_validate_loss:
            min_validate_loss = current_validation_loss
            model.save_weights("./ckpt.h5")

        if no_improvements > PATIENCE:
            used_epochs = epoch
            break
                  

    epoch_losses = []
    for step, batch in enumerate(GENERATOR):
        if step >= TRAIN_STEPS:
            break
        spectrogram, drr, distances, rooms = batch
        model_loss = train_step(spectrogram, drr, distances)
        epoch_losses.append(model_loss)
        print(f"Epoch: {epoch}, Epoch Loss: {round(float(mean_epoch_loss),4)} | Validation Loss: {round(float(current_validation_loss),4)} | Training Loss: {round(float(model_loss),4)}         \r", end="")

if platform.system() == "Windows":
    import winsound
    winsound.PlaySound("SystemHand", winsound.SND_ALIAS)

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
fig = plt.figure(figsize =(20, 7))

if CRITERIA == "median":
    plt.plot(range(0, len(median_train_epoch_losses)), median_train_epoch_losses, color="blue")
    plt.plot(range(0, len(median_validate_epoch_losses)), median_validate_epoch_losses, color="orange")
    plt.axhline(y = np.min(median_validate_epoch_losses), color = "black", linewidth=0.2, linestyle = "dashed")
else:
    plt.plot(range(0, len(mean_train_epoch_losses)), mean_train_epoch_losses, color="blue")
    plt.plot(range(0, len(mean_validate_epoch_losses)), mean_validate_epoch_losses, color="orange")
    plt.axhline(y = np.min(mean_validate_epoch_losses), color = "black", linewidth=0.2, linestyle = "dashed")
plt.show()

In [23]:
# v = 20 best with mish instead of relu (v = 21) maybe best net so far v = 23 mk = 1
v = 24
mk = 1

In [25]:
model.load_weights("./ckpt.h5")

In [26]:
model.save_weights(f"./whistleDistanceNetV{v}Mk{mk}_weights.h5")
model.save(f"./whistleDistanceNetV{v}Mk{mk}_model.h5")

In [27]:
TEST_GENERATOR = data_generator(whistle_length_test, X_TEST, Y_TEST, ROOMS_TEST)
STEPS = len(whistle_length_train[whistle_length_train>=MIN_WHISTLE_SIZE])/BATCH_SIZE

error = []
pred_distances = []
true_distances = []
for step, batch in enumerate(TEST_GENERATOR):
    if step >= STEPS:
            break
    spectrograms, drr, distances, room = batch
    pred_distance = model.predict([spectrograms, drr])
    pred_distances.extend(np.argmax(pred_distance, axis=-1)/(len(BINS)-1))
    true_distances.extend(np.argmax(distances, axis=-1)/(len(BINS)-1))

pred_distance = np.array(pred_distance)
true_distances = np.array(true_distances)

In [ ]:
distance_error = np.abs(true_distances - pred_distances)
print(f"Distance:")
print(f"   min: {np.round(np.min(distance_error), 2)}")
print(f"   5%Q: {np.round(np.quantile(distance_error, 0.05), 2)}")
print(f"median: {np.round(np.median(distance_error), 2)}")
print(f"  mean: {np.round(np.mean(distance_error), 2)}")
print(f"  95%Q: {np.round(np.quantile(distance_error, 0.95), 2)}")
print(f"   max: {np.round(np.max(distance_error), 2)}")
print(f" Range: {np.round(np.quantile(distance_error, 0.95), 2) - np.round(np.quantile(distance_error, 0.05), 2)}")

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (7,7)

fig, axs = plt.subplots(1, 1)
axs.boxplot(np.squeeze(distance_error), showfliers=True)
axs.set_xticklabels(["Distance Error"])
plt.show()